This software will try to load items with different sizes on the a specific rack or container.
And display suggested location for items and putting sequence.

In [76]:
import time
import numpy as np

In [77]:
def IsSpaceEnough(itemSize,itemIndex):
    if itemIndex == 0:
        boundaryX = (len(myMap[0]) - itemSize[0])/2 +1
        boundaryY = (len(myMap) - itemSize[1])/2 +1
    else:
        boundaryX = len(myMap[0]) - itemSize[0] +1
        boundaryY = len(myMap) - itemSize[1] +1
    boundaryZ = ceilingHeight
    
    global emptySpace
    spaceLocated = []
    
    if emptySpace < (itemSize[0] * itemSize[1] * itemSize[2]):
        return spaceLocated
    for posY in range(boundaryY):
        #search every row looking for enough space
        for posX in range(boundaryX):
            if myMap[posY][posX] >boundaryZ - itemSize[2]:
                continue
            if np.array_equal(myMap[posY:(posY + itemSize[1]),posX:(posX + itemSize[0])],np.full((itemSize[1],itemSize[0]),myMap[posY][posX])):
                spaceLocated.append([posY,posX])

    return spaceLocated

In [78]:
def RecursiveLocation(itemIndex,log):
    global emptySpace
    global bestScore
    global ignoreSize
    global retryCount
    global totalSpace
    
    if retryCount == 90:
        return
    #If all items have been evaluated,end the process.
    if itemIndex == len(items):
        if bestScore > emptySpace:
            bestScore = emptySpace
            Fillrate =  float("%.2f" %float((totalSpace - bestScore)*100/float(totalSpace)))
            print "Fill rate of the rack:" ,Fillrate,"%"
            #print log
            print myMap
            print   
        else:
            retryCount = retryCount +1
        return
    
    #find the list of places available for the item 
    locationCandidate = IsSpaceEnough(items[itemIndex],itemIndex)
    #No available pace for current item then move on to next item.
    if len(locationCandidate) == 0:
        RecursiveLocation((itemIndex+1),log)
    else:# If the item is too small, place it in first available location
        if emptySpace/float(totalSpace) < 0.2 + (retryCount /100.0):
            PlaceItem(locationCandidate[0],itemIndex)
            curlog = log + " Item" + str(itemIndex) + " in Loc:" + str(locationCandidate[0]) + " "
            RecursiveLocation((itemIndex+1),curlog)
            RemoveItem(locationCandidate[0],itemIndex)
        else:# loop through all available places for the item.
            for newLocation in locationCandidate:
                PlaceItem(newLocation,itemIndex)
                curlog = log + " Item" + str(itemIndex) + " in Loc:" + str(newLocation)
                RecursiveLocation((itemIndex+1),curlog)
                RemoveItem(newLocation,itemIndex)

In [79]:
def PlaceItem(location,itemIndex):
    global ceilingHeight
    global emptySpace

    emptySpace = emptySpace - (items[itemIndex][0] * items[itemIndex][1] * items[itemIndex][2])
    myMap[location[0]:location[0] + items[itemIndex][1] , location[1]: location[1] + items[itemIndex][0]] += np.full((items[itemIndex][1],items[itemIndex][0]),items[itemIndex][2])

In [80]:
def RemoveItem(location,itemIndex):
    global ceilingHeight
    global emptySpace
    
    emptySpace = emptySpace + (items[itemIndex][0] * items[itemIndex][1] * items[itemIndex][2])
    myMap[location[0]:location[0] + items[itemIndex][1] , location[1]: location[1] + items[itemIndex][0]] -= np.full((items[itemIndex][1],items[itemIndex][0]),items[itemIndex][2])

In [81]:
def PrintMap():
    for Y in range(len(myMap)):
        print myMap[Y]

# Initialize container size (width,length, height)

In [84]:
global bestScore
global retryCount
global ceilingHeight 
global totalSpace
#define size of the container
rackWidth = 10
rackLength = 15
ceilingHeight = 10

#Setup size of rack
myMap = np.zeros((rackLength,rackWidth),dtype = np.int)
print "Height :",ceilingHeight
print myMap

#Initial space the rack has
global emptySpace 
emptySpace= ceilingHeight * len(myMap) * len(myMap[0])
totalSpace = emptySpace

#Put results in a list alone with time needed
timeTaken=[]


Height : 10
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [85]:
print "Total space: ",emptySpace
print

for i in range(1):
    retryCount = 0
    items = np.random.randint(2,5,size = (200,3))

    sec_start = time.time()
    bestScore = emptySpace
    RecursiveLocation(0,"")
    Fillrate =  float("%.2f" %float((totalSpace - bestScore)*100/float(totalSpace)))
    print "Fill rate of the rack:" ,Fillrate,"%"
    timeTaken.append([Fillrate,float("%.2f" % (time.time() - sec_start))])
    print
print "Best result so far :",timeTaken

Total space:  1500

Fill rate of the rack: 78.93 %
[[ 9  9  7  3 10 10  9  9 10 10]
 [ 9  9  7  3 10 10  9  9 10 10]
 [10 10 10 10 10 10  9  9 10 10]
 [10 10 10 10  0  0  9  9 10 10]
 [10 10 10 10  9  9  6 10 10  0]
 [10 10 10 10  9  9  6 10 10  0]
 [ 8  8  8  8  3  3  3 10 10  0]
 [ 9  9 10 10  0  0  0  8  8  0]
 [ 9  9 10 10 10 10 10  6 10 10]
 [ 9  9  6  6 10 10 10  6 10 10]
 [10 10 10  8 10 10 10  3 10 10]
 [10 10 10  8  7  7  7  3 10 10]
 [10 10 10  8  9  9  9  0  9  9]
 [10 10 10  6  9  9  9  0  9  9]
 [ 8  8  8  6  2  2  2  0  0  0]]

Fill rate of the rack: 79.2 %
[[ 9  9  7  3 10 10  9  9 10 10]
 [ 9  9  7  3 10 10  9  9 10 10]
 [10 10 10 10 10 10  9  9 10 10]
 [10 10 10 10  0  0  9  9  8  8]
 [10 10 10 10  9  9  6 10 10  0]
 [10 10 10 10  9  9  6 10 10  0]
 [ 8  8  8  8  3  3  3 10 10  0]
 [ 9  9 10 10  0  0  0 10 10  0]
 [ 9  9 10 10 10 10 10  6 10 10]
 [ 9  9  6  6 10 10 10  6 10 10]
 [10 10 10 10 10 10 10  3 10 10]
 [10 10 10 10  7  7  7  3 10 10]
 [10 10  8  8  9  9  9  0 

The following is quoted from supply chain consultant compnay website:

"In order for a shipment to be acceptable, Contoso relies on a rule of thumb that implies that the container target fill rate should be fixed at roughly 80% of its nominal capacity in order to ensure that all goods fit the container without too much effort. Contoso does not try to leverage exactly 100% of its container capacity because in practice this involves too much complexity for too little return. Instead, Contoso relies on a heuristic that acts as an indicator that a container is approximately full; an indicator which yields rather satisfactory results" 

https://www.lokad.com/salescast-container-shipments